<a href="https://colab.research.google.com/github/bikashk123/Severity-of-an-Airplane-Accident-Using-Machine-Learning/blob/master/severity_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df1=pd.read_csv('train.csv')
df2=pd.read_csv('test.csv')

In [ ]:
df1.head()

,Severity,Safety_Score,Days_Since_Inspection,Total_Safety_Complaints,Control_Metric,Turbulence_In_gforces,Cabin_Temperature,Accident_Type_Code,Max_Elevation,Violations,Adverse_Weather_Metric,Accident_ID
0,Minor_Damage_And_Injuries,49.223744,14,22,71.285324,0.272118,78.04,2,31335.476824,3,0.424352,7570
1,Minor_Damage_And_Injuries,62.465753,10,27,72.288058,0.423939,84.54,2,26024.711057,2,0.352350,12128
2,Significant_Damage_And_Fatalities,63.059361,13,16,66.362808,0.322604,78.86,7,39269.053927,3,0.003364,2181
3,Significant_Damage_And_Serious_Injuries,48.082192,11,9,74.703737,0.337029,81.79,3,42771.499200,1,0.211728,5946
4,Significant_Damage_And_Fatalities,26.484018,13,25,47.948952,0.541140,77.16,3,35509.228515,2,0.176883,9054


In [ ]:
df1['Severity'].value_counts()

Highly_Fatal_And_Damaging                  3049
Significant_Damage_And_Serious_Injuries    2729
Minor_Damage_And_Injuries                  2527
Significant_Damage_And_Fatalities          1695
Name: Severity, dtype: int64

In [ ]:
df1.isnull().any().sum()

0

In [ ]:
df1.columns

Index(['Severity', 'Safety_Score', 'Days_Since_Inspection',
       'Total_Safety_Complaints', 'Control_Metric', 'Turbulence_In_gforces',
       'Cabin_Temperature', 'Accident_Type_Code', 'Max_Elevation',
       'Violations', 'Adverse_Weather_Metric', 'Accident_ID'],
      dtype='object')

In [ ]:
x_train=df1.drop(['Severity','Accident_ID'],axis=1)
y_train=df1['Severity']

In [ ]:
y_train = pd.get_dummies(y_train)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(x_train)

In [ ]:
x_train.shape,y_train.shape

((10000, 10), (10000, 4))

In [ ]:
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier(criterion='entropy',random_state=0,max_features=9,min_samples_leaf=4)
classifier.fit(x_train,y_train)
print("Best score is {}".format(decision_cv.best_score_))  #from random search

NameError: name 'decision_cv' is not defined

In [ ]:
from scipy.stats import randint 
from sklearn.model_selection import RandomizedSearchCV 
param_dist = {"max_depth": [3, None], 
              "max_features": randint(1, 11 ),
              "min_samples_leaf": randint(1, 11),
              "criterion": ["gini", "entropy"]} 
decision_cv = RandomizedSearchCV(classifier, param_dist, cv = 10) 
decision_cv.fit(x_train, y_train)   
print("Tuned Decision Tree Parameters: {}".format(decision_cv.best_params_)) # Print the tuned parameters
print("Best score is {}".format(decision_cv.best_score_)) # Print score 

Tuned Decision Tree Parameters: {'criterion': 'entropy', 'max_depth': None, 'max_features': 7, 'min_samples_leaf': 1}
Best score is 0.9103


In [ ]:
df2.head()

,Safety_Score,Days_Since_Inspection,Total_Safety_Complaints,Control_Metric,Turbulence_In_gforces,Cabin_Temperature,Accident_Type_Code,Max_Elevation,Violations,Adverse_Weather_Metric,Accident_ID
0,19.497717,16,6,72.151322,0.388959,78.32,4,37949.724386,2,0.069692,1
1,58.173516,15,3,64.585232,0.250841,78.60,7,30194.805567,2,0.002777,10
2,33.287671,15,3,64.721969,0.336669,86.96,6,17572.925484,1,0.004316,14
3,3.287671,21,5,66.362808,0.421775,80.86,3,40209.186341,2,0.199990,17
4,10.867580,18,2,56.107566,0.313228,79.22,2,35495.525408,2,0.483696,21


In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier(n_estimators=100,n_jobs=-1,random_state=0)
rf.fit(x_train,y_train)
print("Best score is {}".format(decision_cv.best_score_))

Best score is 0.9103


In [ ]:
x_test = df2.drop(columns=['Accident_ID'])
x_test = sc.transform(x_test)

In [ ]:
y_pred = classifier.predict(x_test)
#y_pred = rf.predict(x_test)

In [ ]:
y_pred

array([[1, 0, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 0, 1],
       ...,
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [1, 0, 0, 0]], dtype=uint8)

In [ ]:
df=pd.DataFrame(data=y_pred,columns=['Highly_Fatal_And_Damaging','Minor_Damage_And_Injuries','Significant_Damage_And_Fatalities',
           'Significant_Damage_And_Serious_Injuries'])

In [ ]:
result=df.idxmax(axis=1)  

In [ ]:
submission=pd.DataFrame(data=df2['Accident_ID'],columns=['Accident_ID','Severity'])
submission['Severity']=result

In [ ]:
submission.to_csv('submission.csv', index=False)